In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
#Clean up code
#add iterative testing
#create test result charts
#output results and charts to folders
#add diagonal to MAE
#create MFE
#find perfect trades to determine optimal moving average

In [3]:
%matplotlib qt
#%matplotlib inline

#import statements 
import xlsxwriter
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
import matplotlib.dates as dates
import datetime
import time
import csv
import sys
import mpl_finance
from mpl_finance import candlestick_ohlc
from IPython import get_ipython
import matplotlib.ticker as mticker
from string import digits

In [4]:

np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})

class Account:
    def __init__(self, name, acct_bal, equity):
        self.name = name
        self.init_bal = acct_bal #initial balance
        self.acct_bal = acct_bal #ongoing account balance
        self.equity = equity #unrealized p/L
        self.buy_status = 1 #these aren't really doing anything
        self.sell_status = 0
        self.trades = 1
        self.profit = 0
        self.units = 0
        self.trade_profit = 0
        #self.activity
        self.activity = np.zeros((1,), dtype=[('Trade', 'int32'), ('Ask', 'float64'), ('Ask_TID', 'int32'), 
                                              ('Units','int32'), ('Bid', 'float64'), ('Bid_TID','int32'), 
                                              ('Profit', 'float64'), ('Stop_Loss_Floor', 'float64'), ('Stop_Loss_Bid', 'float64'), 
                                              ('Stop_Loss_Bid_TID','int32'), ('Max_DD', 'float64'), ('Max_DD_TID', 'int32'), 
                                              ('Max_DD_Bid', 'float64'), ('Stop_Loss_Max_DD', 'float64'), 
                                              ('Stop_Loss_Max_DD_TID', 'int32'),('Stop_Loss_Max_DD_Bid', 'float64')])
        self.new_activity = np.empty_like(self.activity)
        self.buy_start = 0
        self.buy_end = 0
        self.stop_loss = None #define stop loss for reporting
        
        
    def buy(self, amt, arr_analysis_iter):
        #you buy at the ask and sell at the bid (quoted from perspective of market maker)
        
        #pull arr_analysis tuple to assign variable values
        ask = arr_analysis[arr_analysis_iter]['Ask']
        TID = arr_analysis[arr_analysis_iter]['TID']
        self.buy_start = arr_analysis_iter
        
        
        #calculate the buy
        self.units = int((amt)/ask) #calcluate units per trade at amount
        self.equity += (self.units * ask) #set equity to unrealized p/l (this is more accurately "position")
        self.acct_bal -= self.equity #subtract equity from ongoing account balance
        
        #update activity
        self.new_activity[0]['Trade']= self.trades
        self.new_activity[0]['Ask']= ask
        self.new_activity[0]['Ask_TID']= TID
        self.new_activity[0]['Units']= self.units
                
        #set flags
        self.buy_status = 0
        self.sell_status = 1

        
    def sell(self, arr_analysis_iter, stop_loss = None):
        
        self.stop_loss = stop_loss
        
        #pull arr_analysis tuple to assign variable values
        bid = arr_analysis[arr_analysis_iter]['Bid'] #pull the current Bid (what you can sell for)
        TID = arr_analysis[arr_analysis_iter]['TID'] #pull the TID for the current Bid
        self.buy_end = arr_analysis_iter + 1 #Define the end of the array as the current row plus 1 
        
        #determine the maximum drawdown
        ask = self.new_activity['Ask'] #the price you bought at
        ask_tid = self.new_activity['Ask_TID'] #the ID of the tick for the price you bought at
                
        arr_sub_analysis = arr_analysis[self.buy_start:self.buy_end]
        
        min_bid = np.amin(arr_sub_analysis['Bid']) #the lowest purchase price in the range of your purchase to sale
        
        arr_min_bid_tuple = arr_sub_analysis[np.argmin(arr_sub_analysis['Bid'])] #find the index for the lowest bid
        min_bid_tid = arr_min_bid_tuple['TID'] #get the TID for the lowest bid
        
        self.new_activity[0]['Max_DD'] = (min_bid * self.units) - self.equity #calculate the max drawdown amount
        self.new_activity[0]['Max_DD_TID'] = min_bid_tid #pull the TID for the max dd
        self.new_activity[0]['Max_DD_Bid'] = min_bid #pull the bid associated with the max dd
        self.new_activity[0]['Stop_Loss_Max_DD'] = (min_bid * self.units) - self.equity #default stop loss max dd amount for graphing
        self.new_activity[0]['Stop_Loss_Max_DD_TID'] = min_bid_tid #default stop loss max dd tid for graphing
        self.new_activity[0]['Stop_Loss_Max_DD_Bid'] = min_bid #default stop loss max dd bid for graphing

        
        if self.stop_loss != None: #only calculate stop loss values if stop loss is specified        
            stop_loss_floor = (self.equity-self.stop_loss)/self.units #calculate the Bid value of your loss stop
            self.new_activity[0]['Stop_Loss_Floor'] = stop_loss_floor

            #get array of all ticks where bid is gte to stop loss floor
            arr_gte_stop_loss = np.squeeze(np.take(arr_sub_analysis, np.where(arr_sub_analysis['Bid']>= stop_loss_floor)))
                
        if stop_loss == None or stop_loss > abs(self.new_activity[0]['Max_DD']):
            self.new_activity[0]['Bid'] = bid #assign the parameter bid
            self.new_activity[0]['Bid_TID'] = TID #assign the parameter bid TID
            self.new_activity[0]['Stop_Loss_Floor'] = 0
            self.new_activity[0]['Stop_Loss_Bid'] = 0
            self.new_activity[0]['Stop_Loss_Bid_TID'] = 0
            
            self.calc_sale()
                                
        else:
            try:
                if arr_gte_stop_loss.size < 2: #handle data sets where bid is never >= min bid. We actuially want to skip stop loss and just use standard logic for realism
                    self.new_activity[0]['Stop_Loss_Bid'] = 0
                    self.new_activity[0]['Stop_Loss_Bid_TID'] = 0
                    self.new_activity[0]['Bid'] = bid #assign the parameter bid
                    self.new_activity[0]['Bid_TID'] = TID #assign the parameter bid TID
                    self.calc_sale()
                
                else:
                    stop_loss_bid_index = np.argmin(arr_gte_stop_loss['TID'])#get the index of lowest acceptable bid
                    stop_loss_TID = np.amin(arr_gte_stop_loss['TID'])
                    arr_stop_loss_bid = arr_gte_stop_loss[stop_loss_bid_index] #the row of the lowest purchase price in the range of your purchase to sale
                    stop_loss_bid = arr_stop_loss_bid['Bid'] #the ID of the lowest purchase price
                
                    self.new_activity[0]['Stop_Loss_Bid'] = stop_loss_bid
                    self.new_activity[0]['Stop_Loss_Bid_TID'] = stop_loss_TID
                    self.new_activity[0]['Bid'] = stop_loss_bid
                    self.new_activity[0]['Bid_TID'] = stop_loss_TID
                    
                    self.new_activity[0]['Stop_Loss_Max_DD'] = (stop_loss_bid * self.units) - self.equity #calculate the max drawdown amount
                    self.new_activity[0]['Stop_Loss_Max_DD_TID'] = stop_loss_TID #this is actually the lowest bid and not the dd bid
                    self.new_activity[0]['Stop_Loss_Max_DD_Bid'] = stop_loss_bid #this is actually the lowest bid and not the dd bid


                    self.calc_sale()
                            
            except:
                print("Unexpected error:", sys.exc_info())
                #print(err)                
                print("The current trade is:", self.trades)
                print("The current ask is:", self.new_activity[0]['Ask'])
                print("The current equity is:", self.equity)
                print("The current units are:", self.units)
                print("The current min bid is:", min_bid)
                print("The min bid criteria is:", max_dd_criteria)
                print("Array analysis starts at:", self.buy_start, "and ends at:", self.buy_end)
                sys.exit('Exiting because of error')
        
        #calc_sale()
        
        
    def calc_sale(self):
        #calculate the sale
        self.acct_bal = self.acct_bal + (self.units * self.new_activity[0]['Bid']) #add realized p/l to account balance        
        self.trade_amt = self.units * self.new_activity[0]['Bid'] #calculate trade value
        self.trade_profit = (self.units * self.new_activity[0]['Bid']) - self.equity #calculate the trade profit
        self.profit += self.trade_profit #update running profit
        self.equity = 0 #need to change this so it takes bid into accountif stop_loss == None or stop_loss < abs(self.new_activity[0]['Max_DD']):
        
        #update activity
        #self.new_activity[0]['Bid'] = bid
        #self.new_activity[0]['Bid_TID'] = TID
        self.new_activity[0]['Profit']=self.trade_profit
        self.activity = np.append(self.activity, self.new_activity)
        
        #set flags
        self.buy_status = 1
        self.sell_status = 0
        self.trades += 1
    
    def acct_status(self):
        print("The current account balance is:{0:.3f}".format(self.acct_bal), 
              "The current equity is:{0:.3f}".format(self.equity),
              "The profit for this trade was:{0:.3f}".format(self.trade_profit),
              "The current profit is:{0:.3f}".format(self.profit), 
              #"The drawdown for this trade is:{0:.3f}".format(self.activity[0]['Max_DD']),
              self.activity,
              sep='\n')
    
    

In [5]:
class Metrics:
    def __init__(self, sma, fma):
        self.sma = sma
        self.fma = fma
        
    def stat_creation(self):
        #Stat column creation
        df_analysis['Bid_Fast_Avg']=df_analysis.Bid.rolling(self.fma).mean()
        df_analysis['Prev_Bid_Fast_Avg']=df_analysis['Bid_Fast_Avg'].shift()
        df_analysis['Bid_Slow_Avg']=df_analysis.Bid.rolling(self.sma).mean()
        df_analysis['Prev_Bid_Slow_Avg']=df_analysis['Bid_Slow_Avg'].shift()
        df_analysis['Ask_Fast_Avg']=df_analysis.Ask.rolling(self.fma).mean()
        df_analysis['Prev_Ask_Fast_Avg']=df_analysis['Ask_Fast_Avg'].shift()
        df_analysis['Ask_Slow_Avg']=df_analysis.Ask.rolling(self.sma).mean()
        df_analysis['Prev_Ask_Slow_Avg']=df_analysis['Ask_Slow_Avg'].shift()
        df_analysis['Spread_Fast_Avg']=df_analysis.Bid_Ask_Spread.rolling(self.fma).mean()
        df_analysis['Prev_Spread_Fast_Avg']=df_analysis['Spread_Fast_Avg'].shift()
        df_analysis['Spread_Slow_Avg']=df_analysis.Bid_Ask_Spread.rolling(self.sma).mean()
        df_analysis['Prev_Spread_Slow_Avg']=df_analysis['Spread_Slow_Avg'].shift()

        #Final analysis dataset creation
        df_analysis_final=df_analysis.dropna()

        #analysis array creation
        arr_ip = [tuple(i) for i in df_analysis_final.values]

        dtyp = np.dtype(list(zip(df_analysis_final.dtypes.index, df_analysis_final.dtypes)))
        
        global arr_analysis

        arr_analysis = np.array(arr_ip, dtype=dtyp)        

In [6]:
class Reporting:
    def __init__(self):
        ##### build arrays #####
        global arr_results
        arr_results = np.empty((1,), dtype=[('TestName', 'S100'), ('TotalNetProfit', 'float64'), 
                                            ('TotalTrades', 'int'), ('GrossProfit', 'float64'),
                                            ('GrossLoss', 'float64'), ('ProfitFactor', 'float64'),
                                            ('PercentProfitable', 'float64'),('WinningTrades', 'int'),
                                            ('LosingTrades', 'int'), ('EvenTrades', 'int'),
                                            ('AvgTradeNetProfit', 'float64'), ('AvgWinningTrade', 'float64'),
                                            ('AvgLosingTrade', 'float64'),('RatioAvgWinAvgLoss', 'float64'),
                                            ('LargestWinningTrade', 'float64'), ('LargestLosingTrade', 'float64'),
                                            ('MaxConWinTrade', 'int'),('MaxConLoseTrade', 'int'),
                                            ('AvgBarsInTotalTrades', 'float64'), ('AvgBarsInWinTrades', 'float64'),
                                            ('AvgBarsInLosTrades', 'float64')
                                           ])
        
        global arr_new_results

        arr_new_results = np.empty_like(arr_results)
        
        
    def populate_results(self):
        trades = trading_test.activity[1:,]

        positive_trades = np.squeeze(np.take(trades, np.where(trades['Profit']>0)))
        negative_trades = np.squeeze(np.take(trades, np.where(trades['Profit']<0)))
        even_trades = np.squeeze(np.take(trades, np.where(trades['Profit']==0)))
        
        ###### get longest winning and losing trade streak ######

        iterator = 0
        curr_max_con_win_trade = 0
        max_con_win_trade = 0

        for i in np.ndenumerate(positive_trades['Trade']):
            if iterator != 0:
                prev_trade = iterator - 1
                if positive_trades[prev_trade]['Trade'] + 1 == positive_trades[iterator]['Trade']:
                    curr_max_con_win_trade += 1
                    if curr_max_con_win_trade > max_con_win_trade:
                        max_con_win_trade = curr_max_con_win_trade
                else:
                    curr_max_con_win_trade = 0

            iterator += 1


        iterator = 0
        curr_max_con_lose_trade = 0
        max_con_lose_trade = 0

        for i in np.ndenumerate(negative_trades['Trade']):
            if iterator != 0:
                prev_trade = iterator - 1
                if negative_trades[prev_trade]['Trade'] + 1 == negative_trades[iterator]['Trade']:
                    curr_max_con_lose_trade += 1
                    if curr_max_con_lose_trade > max_con_lose_trade:
                        max_con_lose_trade = curr_max_con_lose_trade
                else:
                    curr_max_con_lose_trade = 0

            iterator += 1
        
        ##### populate arr_new_results #####
        arr_new_results['TestName'] = trading_test.name
        arr_new_results['TotalNetProfit'] = trading_test.profit
        arr_new_results['TotalTrades'] = trades.size #have to subtract the first 0 rowfrom np.zero
        arr_new_results['GrossProfit'] = positive_trades['Profit'].sum()
        arr_new_results['GrossLoss'] = negative_trades['Profit'].sum()
        arr_new_results['ProfitFactor'] = positive_trades['Profit'].sum()/negative_trades['Profit'].sum()
        arr_new_results['PercentProfitable'] = positive_trades.size/(trades.size)
        arr_new_results['WinningTrades'] = positive_trades.size
        arr_new_results['LosingTrades'] = negative_trades.size
        arr_new_results['EvenTrades'] = even_trades.size
        arr_new_results['AvgTradeNetProfit'] = trading_test.profit/trades.size
        arr_new_results['AvgWinningTrade'] = positive_trades['Profit'].sum()/positive_trades.size
        arr_new_results['AvgLosingTrade'] = negative_trades['Profit'].sum()/negative_trades.size
        arr_new_results['RatioAvgWinAvgLoss'] = (positive_trades['Profit'].sum()/positive_trades.size)/(negative_trades['Profit'].sum()/negative_trades.size)
        arr_new_results['LargestWinningTrade'] = np.amax(positive_trades['Profit']) if positive_trades.size > 0 else 0
        arr_new_results['LargestLosingTrade'] = np.amin(negative_trades['Profit']) if negative_trades.size > 0 else 0
        arr_new_results['MaxConWinTrade'] = max_con_win_trade
        arr_new_results['MaxConLoseTrade'] = max_con_lose_trade
        arr_new_results['AvgBarsInTotalTrades'] = (trades['Bid_TID'].sum() - trades['Ask_TID'].sum())/trades.size
        arr_new_results['AvgBarsInWinTrades'] = (positive_trades['Bid_TID'].sum() - positive_trades['Ask_TID'].sum())/positive_trades.size
        arr_new_results['AvgBarsInLosTrades'] = (negative_trades['Bid_TID'].sum() - negative_trades['Ask_TID'].sum())/negative_trades.size
        
        global arr_results
        
        arr_results = np.append(arr_results, arr_new_results)        
        
        #Maximum Adverse Event
        f, ax = plt.subplots(figsize=(20,10))
        ax.scatter(abs(positive_trades['Stop_Loss_Max_DD']), abs(positive_trades['Profit']), marker="+", c="g")
        ax.scatter(abs(negative_trades['Stop_Loss_Max_DD']), abs(negative_trades['Profit']), marker="o", c="r")

        ax.plot(min(ax.get_ylim(),ax.get_xlim()) , min(ax.get_ylim(),ax.get_xlim()), ls="--", c=".3")
        ax.grid(color='g', linestyle='dashed', linewidth=1)
        ax.set_xlim(xmin=0)
        ax.set_ylim(ymin=0)
        if trading_test.stop_loss != None: plt.axvline(x=trading_test.stop_loss)
        plt.xlabel('Max Drawdown')
        plt.ylabel('Profit ($)')
        ax.set_title("Maximum Adverse Event \n Stop Loss:" +str(trading_test.stop_loss))
        plt.savefig('C:\\Users\\pebaqu\\Desktop\Personal\\Python\\JupyterExports\\'+trading_test.name+'.png')
        
        
    def create_final_report(self):
        #chart the results
        
        global df_results
        
        df_results = pd.DataFrame(data=arr_results,    # values
                    columns=arr_results.dtype.names)  # 1st row as the column names

        df_results.index = df_results['TestName']
        
        global df_final

        df_final = df_results.iloc[1:,1:].T

        display(df_final)
        

In [7]:
%%time

#Current anlayis code

USD_JPY_df =  pd.read_pickle("C:/Users/pebaqu/OneDrive - SAS/Profiles for l10c581/l10c581/Desktop/Personal/Python/Datasets/2018-07-01_USD_JPY")

df_usd_jpy = USD_JPY_df.drop(['Complete','Time', 'Volume'], axis=1).copy()
df_usd_jpy.reset_index(inplace=True)
df_usd_jpy['TID']= df_usd_jpy.index
length = max(df_usd_jpy['TID']) + 1

df_usd_jpy['BidAvg']= df_usd_jpy[['Bid_Open','Bid_High','Bid_Low','Bid_Close']].mean(axis=1)
df_usd_jpy['AskAvg']= df_usd_jpy[['Ask_Open','Ask_High','Ask_Low','Ask_Close']].mean(axis=1)



slow_moving_percents = [.05, .10, .15, .20, .25, .30, .35, .40, .45, .50]
fast_moving_percents = [.05, .10, .15, .20, .25, .30, .35, .40, .45, .50]

slow_ma = [int(i * length) for i in slow_moving_percents] 


for sma in slow_ma:
    df_usd_jpy['Bid_SSMA'+str(sma)]= df_usd_jpy['Bid_Close'].rolling(sma).mean()
    #print(sma, [int(sma*fma) for fma in fast_moving_percents])
    for fma in fast_moving_percents:
        fma_window = int(sma*fma)
        df_usd_jpy['Bid_FSMA'+str(fma_window)]= df_usd_jpy['Bid_Close'].rolling(fma_window).mean()

df_usd_jpy.rename(columns={'BidAvg':'Bid', 'AskAvg':'Ask'}, inplace = True)

arr_ip = [tuple(i) for i in df_usd_jpy.values]

dtyp = np.dtype(list(zip(df_usd_jpy.dtypes.index, df_usd_jpy.dtypes)))
        
global arr_analysis

arr_analysis = np.array(arr_ip, dtype=dtyp)

report = Reporting()
plt.ioff()

iterator = 0

for sma in slow_ma:    

    stop_loss = [None, .25, .75, 1]

    stop_loss_iter = 0
    
    fma_val = [int(sma*fma) for fma in fast_moving_percents]
    
    for z in range(len(stop_loss)):
        
        for fma in fma_val:
            trading_test = Account('test'+str(iterator)+"Bid_SSMA"+str(sma)+"Bid_FSMA"+str(fma)+"SL"+str(stop_loss[stop_loss_iter]),50000,0)
            arr_analysis_iter = 0
            
            for x in np.ndenumerate(arr_analysis):

                if (arr_analysis[arr_analysis_iter]['Bid_SSMA'+str(sma)] < arr_analysis[arr_analysis_iter]['Bid_FSMA'+str(fma)] and trading_test.buy_status == 1):
                    trading_test.buy(1000, arr_analysis_iter) 

                elif (arr_analysis[arr_analysis_iter]['Bid_SSMA'+str(sma)] > arr_analysis[arr_analysis_iter]['Bid_FSMA'+str(fma)] and trading_test.sell_status == 1):    
                    trading_test.sell(arr_analysis_iter, stop_loss[stop_loss_iter]) 

                arr_analysis_iter += 1

            report.populate_results()
            iterator += 1

        stop_loss_iter += 1

report.create_final_report()


C:\Users\pebaqu\AppData\Roaming\Python\Python36\site-packages\ipykernel_launcher.py:77: RuntimeWarning: invalid value encountered in double_scalars
C:\Users\pebaqu\AppData\Roaming\Python\Python36\site-packages\ipykernel_launcher.py:79: RuntimeWarning: invalid value encountered in double_scalars
C:\Users\pebaqu\AppData\Roaming\Python\Python36\site-packages\ipykernel_launcher.py:85: RuntimeWarning: invalid value encountered in long_scalars
C:\Users\pebaqu\AppData\Local\Continuum\anaconda3\lib\site-packages\matplotlib\pyplot.py:537: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)
C:\Users\pebaqu\AppData\Roaming\Python\Python36\site-packages\ipykernel_launcher.py:71: RuntimeWarning: divide by zero encountered in double_scalars
C:\Users\p

TestName,b'test0Bid_SSMA6910Bid_FSMA345SLNone',b'test1Bid_SSMA6910Bid_FSMA691SLNone',b'test2Bid_SSMA6910Bid_FSMA1036SLNone',b'test3Bid_SSMA6910Bid_FSMA1382SLNone',b'test4Bid_SSMA6910Bid_FSMA1727SLNone',b'test5Bid_SSMA6910Bid_FSMA2073SLNone',b'test6Bid_SSMA6910Bid_FSMA2418SLNone',b'test7Bid_SSMA6910Bid_FSMA2764SLNone',b'test8Bid_SSMA6910Bid_FSMA3109SLNone',b'test9Bid_SSMA6910Bid_FSMA3455SLNone',...,b'test390Bid_SSMA69100Bid_FSMA3455SL1',b'test391Bid_SSMA69100Bid_FSMA6910SL1',b'test392Bid_SSMA69100Bid_FSMA10365SL1',b'test393Bid_SSMA69100Bid_FSMA13820SL1',b'test394Bid_SSMA69100Bid_FSMA17275SL1',b'test395Bid_SSMA69100Bid_FSMA20730SL1',b'test396Bid_SSMA69100Bid_FSMA24185SL1',b'test397Bid_SSMA69100Bid_FSMA27640SL1',b'test398Bid_SSMA69100Bid_FSMA31095SL1',b'test399Bid_SSMA69100Bid_FSMA34550SL1'
TotalNetProfit,-1.913000,-2.111250,3.365500,-0.089750,-0.107000,-1.370250,-3.593750,-1.033500,0.012750,-0.679500,...,-0.1,-0.1,-0.1,-0.1,-0.1,-0.1,-0.1,-0.1,-0.1,-0.1
TotalTrades,22.000000,18.000000,13.000000,12.000000,12.000000,11.000000,11.000000,11.000000,10.000000,10.000000,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
GrossProfit,15.427000,16.217250,14.651000,12.086000,12.064000,12.490000,10.558500,11.921500,12.798500,13.607500,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
GrossLoss,-17.340000,-18.328500,-11.285500,-12.175750,-12.171000,-13.860250,-14.152250,-12.955000,-12.785750,-14.287000,...,-0.1,-0.1,-0.1,-0.1,-0.1,-0.1,-0.1,-0.1,-0.1,-0.1
ProfitFactor,-0.889677,-0.884811,-1.298215,-0.992629,-0.991209,-0.901138,-0.746065,-0.920224,-1.000997,-0.952439,...,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0
PercentProfitable,0.181818,0.166667,0.307692,0.250000,0.250000,0.272727,0.272727,0.363636,0.400000,0.400000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
WinningTrades,4.000000,3.000000,4.000000,3.000000,3.000000,3.000000,3.000000,4.000000,4.000000,4.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
LosingTrades,18.000000,15.000000,9.000000,8.000000,9.000000,8.000000,8.000000,7.000000,6.000000,6.000000,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
EvenTrades,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
AvgTradeNetProfit,-0.086955,-0.117292,0.258885,-0.007479,-0.008917,-0.124568,-0.326705,-0.093955,0.001275,-0.067950,...,-0.1,-0.1,-0.1,-0.1,-0.1,-0.1,-0.1,-0.1,-0.1,-0.1


Wall time: 7min 8s


In [60]:
#prep dataset for reporting


df_byte_string = df_results.iloc[1:].reset_index(drop=True).copy()

df_byte_string['TestName'] = df_byte_string['TestName'].str.decode('utf-8')

df_byte_string['Test'], df_byte_string['StopLoss'] = df_byte_string['TestName'].str.split('SL', 1).str

df_byte_string['StopLoss_New'] = df_byte_string['StopLoss'].str.replace('None','0')

df_byte_string['Test_Original'] = df_byte_string['TestName']

df_byte_string['TestName_Strip'] = df_byte_string['Test'].str.replace('test','')

df_byte_string['TestName_New'] = df_byte_string['TestName_Strip'].str.lstrip(digits)

df_byte_string.sort_values(by=['TestName_New', 'StopLoss_New'], inplace = True)

display(df_byte_string)



,TestName,TotalNetProfit,TotalTrades,GrossProfit,GrossLoss,ProfitFactor,PercentProfitable,WinningTrades,LosingTrades,EvenTrades,...,MaxConLoseTrade,AvgBarsInTotalTrades,AvgBarsInWinTrades,AvgBarsInLosTrades,Test,StopLoss,StopLoss_New,Test_Original,TestName_Strip,TestName_New
41,test41Bid_SSMA13820Bid_FSMA1382SLNone,3.95925,8,14.31000,-10.35075,-1.382509,0.125000,1,7,0,...,5,7593.000000,25850.000000,4984.857143,test41Bid_SSMA13820Bid_FSMA1382,None,0,test41Bid_SSMA13820Bid_FSMA1382SLNone,41Bid_SSMA13820Bid_FSMA1382,Bid_SSMA13820Bid_FSMA1382
51,test51Bid_SSMA13820Bid_FSMA1382SL0.25,-1.00400,8,0.00000,-1.00400,-0.000000,0.000000,0,8,0,...,7,2.750000,NaN,2.750000,test51Bid_SSMA13820Bid_FSMA1382,0.25,0.25,test51Bid_SSMA13820Bid_FSMA1382SL0.25,51Bid_SSMA13820Bid_FSMA1382,Bid_SSMA13820Bid_FSMA1382
61,test61Bid_SSMA13820Bid_FSMA1382SL0.75,13.18700,8,14.31000,-1.12300,-12.742654,0.125000,1,7,0,...,5,3231.250000,25850.000000,0.000000,test61Bid_SSMA13820Bid_FSMA1382,0.75,0.75,test61Bid_SSMA13820Bid_FSMA1382SL0.75,61Bid_SSMA13820Bid_FSMA1382,Bid_SSMA13820Bid_FSMA1382
71,test71Bid_SSMA13820Bid_FSMA1382SL1,13.18300,8,14.31000,-1.12700,-12.697427,0.125000,1,7,0,...,5,3593.375000,25850.000000,413.857143,test71Bid_SSMA13820Bid_FSMA1382,1,1,test71Bid_SSMA13820Bid_FSMA1382SL1,71Bid_SSMA13820Bid_FSMA1382,Bid_SSMA13820Bid_FSMA1382
42,test42Bid_SSMA13820Bid_FSMA2073SLNone,3.81450,6,14.88150,-11.06700,-1.344673,0.166667,1,5,0,...,3,9916.666667,26150.000000,6670.000000,test42Bid_SSMA13820Bid_FSMA2073,None,0,test42Bid_SSMA13820Bid_FSMA2073SLNone,42Bid_SSMA13820Bid_FSMA2073,Bid_SSMA13820Bid_FSMA2073
52,test52Bid_SSMA13820Bid_FSMA2073SL0.25,-0.67900,6,0.00000,-0.67900,-0.000000,0.000000,0,6,0,...,5,0.000000,NaN,0.000000,test52Bid_SSMA13820Bid_FSMA2073,0.25,0.25,test52Bid_SSMA13820Bid_FSMA2073SL0.25,52Bid_SSMA13820Bid_FSMA2073,Bid_SSMA13820Bid_FSMA2073
62,test62Bid_SSMA13820Bid_FSMA2073SL0.75,14.33750,6,14.88150,-0.54400,-27.355699,0.166667,1,5,0,...,3,4358.333333,26150.000000,0.000000,test62Bid_SSMA13820Bid_FSMA2073,0.75,0.75,test62Bid_SSMA13820Bid_FSMA2073SL0.75,62Bid_SSMA13820Bid_FSMA2073,Bid_SSMA13820Bid_FSMA2073
72,test72Bid_SSMA13820Bid_FSMA2073SL1,14.33750,6,14.88150,-0.54400,-27.355699,0.166667,1,5,0,...,3,4358.333333,26150.000000,0.000000,test72Bid_SSMA13820Bid_FSMA2073,1,1,test72Bid_SSMA13820Bid_FSMA2073SL1,72Bid_SSMA13820Bid_FSMA2073,Bid_SSMA13820Bid_FSMA2073
43,test43Bid_SSMA13820Bid_FSMA2764SLNone,5.35925,6,14.31000,-8.95075,-1.598749,0.166667,1,5,0,...,3,9878.000000,26551.000000,6543.400000,test43Bid_SSMA13820Bid_FSMA2764,None,0,test43Bid_SSMA13820Bid_FSMA2764SLNone,43Bid_SSMA13820Bid_FSMA2764,Bid_SSMA13820Bid_FSMA2764
53,test53Bid_SSMA13820Bid_FSMA2764SL0.25,-0.76000,6,0.00000,-0.76000,-0.000000,0.000000,0,6,0,...,5,0.000000,NaN,0.000000,test53Bid_SSMA13820Bid_FSMA2764,0.25,0.25,test53Bid_SSMA13820Bid_FSMA2764SL0.25,53Bid_SSMA13820Bid_FSMA2764,Bid_SSMA13820Bid_FSMA2764


In [110]:
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import numpy as np

#fig = plt.figure()
#ax = plt.axes()

arr_graph_all = df_byte_string[['TotalNetProfit', 'StopLoss_New','TestName_New']].reset_index().values


pd_graph_all = df_byte_string[['TotalNetProfit', 'StopLoss_New','TestName_New']]


#pd_graph_all.set_index('StopLoss_New', inplace = True)

#display(pd_graph_all)

pd_graph = pd_graph_all

arr_graph = arr_graph_all[:8]

#display(pd_graph)

fig, ax = plt.subplots(figsize=(8,6))
for label, df in pd_graph.groupby('TestName_New'):    
    x = df['StopLoss_New']
    y = df['TotalNetProfit']
    plt.plot(x, y,label=label)
plt.legend()
plt.show()

In [70]:
x1 = np.random.randint(10, size=6)  # One-dimensional array
x2 = np.random.randint(10, size=(3, 4))  # Two-dimensional array
x3 = np.random.randint(10, size=(3, 4, 5))  # Three-dimensional array


print("x1 ndim: ", x1.ndim)
print("x1 shape:", x1.shape)
print("x1 size: ", x1.size)

print("x3 ndim: ", x3.ndim)
print("x3 shape:", x3.shape)
print("x3 size: ", x3.size)

x1 ndim:  1
x1 shape: (6,)
x1 size:  6
x3 ndim:  3
x3 shape: (3, 4, 5)
x3 size:  60


In [99]:
classes = ["class 1"] * 5 + ["class 2"] * 5
vals = [1,3,5,1,3] + [2,6,7,5,2]
p_df = pd.DataFrame({"class": classes, "vals": vals})

fig, ax = plt.subplots(figsize=(8,6))
for label, df in p_df.groupby('class'):
    df.vals.plot(kind="kde", ax=ax, label=label)
plt.legend()
plt.show()

In [100]:
display(p_df)

,class,vals
0,class 1,1
1,class 1,3
2,class 1,5
3,class 1,1
4,class 1,3
5,class 2,2
6,class 2,6
7,class 2,7
8,class 2,5
9,class 2,2


In [19]:
#df.set_index('Date', inplace=True)

#df_results['Test'], df_results['StopLoss'] = df_results['TestName'].str.split('SL', 1).str

df_results.reset_index(drop = True, inplace = True)

display(df_results)



,TestName,TotalNetProfit,TotalTrades,GrossProfit,GrossLoss,ProfitFactor,PercentProfitable,WinningTrades,LosingTrades,EvenTrades,...,AvgWinningTrade,AvgLosingTrade,RatioAvgWinAvgLoss,LargestWinningTrade,LargestLosingTrade,MaxConWinTrade,MaxConLoseTrade,AvgBarsInTotalTrades,AvgBarsInWinTrades,AvgBarsInLosTrades
0,b'',0.00000,0,0.00000,0.00000,0.000000,0.000000,0,0,0,...,0.000000,0.000000,0.000000,0.0000,0.0000,0,0,0.000000,0.000000,0.000000
1,b'test0Bid_SSMA6910Bid_FSMA345SLNone',-1.91300,22,15.42700,-17.34000,-0.889677,0.181818,4,18,0,...,3.856750,-0.963333,-4.003547,10.0240,-3.3920,0,4,2860.500000,7670.000000,1791.722222
2,b'test1Bid_SSMA6910Bid_FSMA691SLNone',-2.11125,18,16.21725,-18.32850,-0.884811,0.166667,3,15,0,...,5.405750,-1.221900,-4.424053,11.0720,-4.6300,0,8,3504.666667,8869.333333,2431.733333
3,b'test2Bid_SSMA6910Bid_FSMA1036SLNone',3.36550,13,14.65100,-11.28550,-1.298215,0.307692,4,9,0,...,3.662750,-1.253944,-2.920983,10.6880,-4.3820,1,4,4872.538462,6866.000000,3986.555556
4,b'test3Bid_SSMA6910Bid_FSMA1382SLNone',-0.08975,12,12.08600,-12.17575,-0.992629,0.250000,3,8,1,...,4.028667,-1.521969,-2.647010,10.4920,-5.0760,0,4,5307.833333,9232.666667,3334.875000
5,b'test4Bid_SSMA6910Bid_FSMA1727SLNone',-0.10700,12,12.06400,-12.17100,-0.991209,0.250000,3,9,0,...,4.021333,-1.352333,-2.973626,9.4360,-6.6320,1,6,5370.750000,10483.666667,3666.444444
6,b'test5Bid_SSMA6910Bid_FSMA2073SLNone',-1.37025,11,12.49000,-13.86025,-0.901138,0.272727,3,8,0,...,4.163333,-1.732531,-2.403035,8.8200,-6.6860,1,4,5878.727273,9551.333333,4501.500000
7,b'test6Bid_SSMA6910Bid_FSMA2418SLNone',-3.59375,11,10.55850,-14.15225,-0.746065,0.272727,3,8,0,...,3.519500,-1.769031,-1.989507,8.5160,-6.6600,1,4,5849.909091,9617.666667,4437.000000
8,b'test7Bid_SSMA6910Bid_FSMA2764SLNone',-1.03350,11,11.92150,-12.95500,-0.920224,0.363636,4,7,0,...,2.980375,-1.850714,-1.610392,8.6320,-6.3120,2,4,5861.454545,9893.500000,3557.428571
9,b'test8Bid_SSMA6910Bid_FSMA3109SLNone',0.01275,10,12.79850,-12.78575,-1.000997,0.400000,4,6,0,...,3.199625,-2.130958,-1.501496,8.7360,-5.5720,3,4,6461.400000,10265.500000,3925.333333


In [30]:
%%time

metric_iter = 0
sma_val = [10000,10000,12000,12000,15000,15000,18000,18000,20000,20000]
fma_val = [2000, 5000, 3000, 6000, 4000, 7500, 6000, 9000, 4000, 10000]

report = Reporting()

for y in range(len(sma_val)):
    metrics = Metrics(sma_val[metric_iter], fma_val[metric_iter])
    metrics.stat_creation()    
    
    stop_loss = [None, .25, .75, 1]
    
    stop_loss_iter = 0
    for z in range(len(stop_loss)):
        trading_test = Account('test'+str(metric_iter)+"SMA"+str(sma_val[metric_iter])+"FMA"+str(fma_val[metric_iter])+"SL"+str(stop_loss[stop_loss_iter]),50000,0)

        arr_analysis_iter = 0
        for x in np.ndenumerate(arr_analysis):

            if (arr_analysis[arr_analysis_iter]['Bid_Fast_Avg'] > arr_analysis[arr_analysis_iter]['Ask_Slow_Avg'] and trading_test.buy_status == 1):
                trading_test.buy(1000, arr_analysis_iter) 

            elif (arr_analysis[arr_analysis_iter]['Bid_Fast_Avg'] < arr_analysis[arr_analysis_iter]['Ask_Slow_Avg'] and trading_test.sell_status == 1):    
                trading_test.sell(arr_analysis_iter, stop_loss[stop_loss_iter]) 

            arr_analysis_iter += 1

        report.populate_results()
        
        stop_loss_iter += 1
        
    metric_iter += 1

report.create_final_report()



C:\Users\pebaqu\AppData\Local\Continuum\anaconda3\lib\site-packages\matplotlib\pyplot.py:537: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


TestName,b'test0SMA10000FMA2000SLNone',b'test0SMA10000FMA2000SL0.25',b'test0SMA10000FMA2000SL0.75',b'test0SMA10000FMA2000SL1',b'test1SMA10000FMA5000SLNone',b'test1SMA10000FMA5000SL0.25',b'test1SMA10000FMA5000SL0.75',b'test1SMA10000FMA5000SL1',b'test2SMA12000FMA3000SLNone',b'test2SMA12000FMA3000SL0.25',...,b'test7SMA18000FMA9000SL0.75',b'test7SMA18000FMA9000SL1',b'test8SMA20000FMA4000SLNone',b'test8SMA20000FMA4000SL0.25',b'test8SMA20000FMA4000SL0.75',b'test8SMA20000FMA4000SL1',b'test9SMA20000FMA10000SLNone',b'test9SMA20000FMA10000SL0.25',b'test9SMA20000FMA10000SL0.75',b'test9SMA20000FMA10000SL1'
TotalNetProfit,-31.861000,18.664000,16.612000,1.437000,-24.515000,17.540000,27.094000,17.584000,-29.317000,12.285000,...,15.630000,15.573000,-17.771000,21.379000,32.643000,20.215000,-7.599000,13.902000,27.245000,31.023000
TotalTrades,230.000000,230.000000,230.000000,230.000000,180.000000,180.000000,180.000000,180.000000,175.000000,175.000000,...,95.000000,95.000000,113.000000,113.000000,113.000000,113.000000,88.000000,88.000000,88.000000,88.000000
GrossProfit,57.327000,26.085000,51.963000,57.327000,49.810000,23.450000,44.446000,48.343000,49.865000,17.827000,...,24.043000,30.134000,46.467000,25.142000,43.713000,43.713000,44.217000,16.488000,32.742000,40.050000
GrossLoss,-89.188000,-7.421000,-35.351000,-55.890000,-74.325000,-5.910000,-17.352000,-30.759000,-79.182000,-5.542000,...,-8.413000,-14.561000,-64.238000,-3.763000,-11.070000,-23.498000,-51.816000,-2.586000,-5.497000,-9.027000
ProfitFactor,-0.642766,-3.515025,-1.469916,-1.025711,-0.670165,-3.967851,-2.561434,-1.571670,-0.629752,-3.216709,...,-2.857839,-2.069501,-0.723357,-6.681371,-3.948780,-1.860286,-0.853346,-6.375870,-5.956340,-4.436690
PercentProfitable,0.291304,0.152174,0.282609,0.291304,0.316667,0.150000,0.294444,0.311111,0.285714,0.102857,...,0.200000,0.242105,0.247788,0.106195,0.221239,0.221239,0.375000,0.136364,0.284091,0.340909
WinningTrades,67.000000,35.000000,65.000000,67.000000,57.000000,27.000000,53.000000,56.000000,50.000000,18.000000,...,19.000000,23.000000,28.000000,12.000000,25.000000,25.000000,33.000000,12.000000,25.000000,30.000000
LosingTrades,163.000000,194.000000,165.000000,163.000000,122.000000,151.000000,124.000000,121.000000,125.000000,156.000000,...,75.000000,71.000000,85.000000,100.000000,87.000000,87.000000,55.000000,74.000000,62.000000,58.000000
EvenTrades,0.000000,1.000000,0.000000,0.000000,1.000000,2.000000,3.000000,3.000000,0.000000,1.000000,...,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,0.000000,2.000000,1.000000,0.000000
AvgTradeNetProfit,-0.138526,0.081148,0.072226,0.006248,-0.136194,0.097444,0.150522,0.097689,-0.167526,0.070200,...,0.164526,0.163926,-0.157265,0.189195,0.288876,0.178894,-0.086352,0.157977,0.309602,0.352534


Wall time: 15min 11s


In [ ]:
#Begin Analysis Section

In [25]:
print(df_final)

NameError: name 'df_final' is not defined

In [8]:
df_analysis['TickCount'] = 1
display(df_analysis)
df_analysis['Seconds'] = pd.to_datetime(df_analysis['TTime'].dt.second, unit = 's')
df_analysis.index = df_analysis['Seconds']
#print(df_analysis.dtypes)
tick_summary = pd.DataFrame()
tick_summary['AvgTickPerSecond'] = df_analysis.TickCount.resample('S').mean()
tick_summary['SumTickPerSecond'] = df_analysis.TickCount.resample('S').sum()

df_analysis['Minutes'] = pd.to_datetime(df_analysis['TTime'].dt.minute, unit = 'm')
df_analysis.index = df_analysis['Minutes']

tick_summary['AvgTickPerMinute'] = df_analysis.TickCount.resample('1Min').mean()
tick_summary['SumTickPerMinute'] = df_analysis.TickCount.resample('1Min').sum()


display(tick_summary)
display(df_analysis)

,Pair,TTime,Bid,Ask,TID,Bid_Ask_Spread,TickCount,Seconds
Seconds,,,,,,,,
1970-01-01 00:00:30,USD/JPY,2018-01-01 22:01:30.499,112.570,112.800,0,-0.230,1,1970-01-01 00:00:30
1970-01-01 00:00:32,USD/JPY,2018-01-01 22:01:32.143,112.600,112.800,1,-0.200,1,1970-01-01 00:00:32
1970-01-01 00:00:50,USD/JPY,2018-01-01 22:01:50.876,112.614,112.722,2,-0.108,1,1970-01-01 00:00:50
1970-01-01 00:00:02,USD/JPY,2018-01-01 22:02:02.819,112.600,112.800,3,-0.200,1,1970-01-01 00:00:02
1970-01-01 00:00:04,USD/JPY,2018-01-01 22:02:04.139,112.614,112.723,4,-0.109,1,1970-01-01 00:00:04
1970-01-01 00:00:22,USD/JPY,2018-01-01 22:02:22.881,112.600,112.800,5,-0.200,1,1970-01-01 00:00:22
1970-01-01 00:00:47,USD/JPY,2018-01-01 22:03:47.489,112.613,112.714,6,-0.101,1,1970-01-01 00:00:47
1970-01-01 00:00:48,USD/JPY,2018-01-01 22:03:48.641,112.610,112.696,7,-0.086,1,1970-01-01 00:00:48
1970-01-01 00:00:02,USD/JPY,2018-01-01 22:04:02.814,112.605,112.695,8,-0.090,1,1970-01-01 00:00:02


,AvgTickPerSecond,SumTickPerSecond,AvgTickPerMinute,SumTickPerMinute
Seconds,,,,
1970-01-01 00:00:00,1,72727,1.0,73220.0
1970-01-01 00:00:01,1,64464,NaN,NaN
1970-01-01 00:00:02,1,59636,NaN,NaN
1970-01-01 00:00:03,1,57359,NaN,NaN
1970-01-01 00:00:04,1,56226,NaN,NaN
1970-01-01 00:00:05,1,55039,NaN,NaN
1970-01-01 00:00:06,1,55229,NaN,NaN
1970-01-01 00:00:07,1,54914,NaN,NaN
1970-01-01 00:00:08,1,53784,NaN,NaN


,Pair,TTime,Bid,Ask,TID,Bid_Ask_Spread,TickCount,Seconds,Minutes
Minutes,,,,,,,,,
1970-01-01 00:01:00,USD/JPY,2018-01-01 22:01:30.499,112.570,112.800,0,-0.230,1,1970-01-01 00:00:30,1970-01-01 00:01:00
1970-01-01 00:01:00,USD/JPY,2018-01-01 22:01:32.143,112.600,112.800,1,-0.200,1,1970-01-01 00:00:32,1970-01-01 00:01:00
1970-01-01 00:01:00,USD/JPY,2018-01-01 22:01:50.876,112.614,112.722,2,-0.108,1,1970-01-01 00:00:50,1970-01-01 00:01:00
1970-01-01 00:02:00,USD/JPY,2018-01-01 22:02:02.819,112.600,112.800,3,-0.200,1,1970-01-01 00:00:02,1970-01-01 00:02:00
1970-01-01 00:02:00,USD/JPY,2018-01-01 22:02:04.139,112.614,112.723,4,-0.109,1,1970-01-01 00:00:04,1970-01-01 00:02:00
1970-01-01 00:02:00,USD/JPY,2018-01-01 22:02:22.881,112.600,112.800,5,-0.200,1,1970-01-01 00:00:22,1970-01-01 00:02:00
1970-01-01 00:03:00,USD/JPY,2018-01-01 22:03:47.489,112.613,112.714,6,-0.101,1,1970-01-01 00:00:47,1970-01-01 00:03:00
1970-01-01 00:03:00,USD/JPY,2018-01-01 22:03:48.641,112.610,112.696,7,-0.086,1,1970-01-01 00:00:48,1970-01-01 00:03:00
1970-01-01 00:04:00,USD/JPY,2018-01-01 22:04:02.814,112.605,112.695,8,-0.090,1,1970-01-01 00:00:02,1970-01-01 00:04:00


In [20]:
#df_analysis.groupby(['Minutes'])['TickCount'].sum()
#display(df_analysis)

df_new_analysis = df_analysis[['Minutes','Seconds','TickCount']]
display(df_new_analysis)

df_group = df_new_analysis.groupby(['Minutes','Seconds']).count()
display(df_group)

df_export = df_group

writer = pd.ExcelWriter('C:\\Users\\pebaqu\\Desktop\\Personal\\Python\\JupyterExports\\TickSampling.xlsx', engine='xlsxwriter')

#Convert the dataframe to an XlsxWriter Excel object.
df_export.to_excel(writer, sheet_name='Sheet1')
#Close the Pandas Excel writer and output the Excel file.
writer.save()


,Minutes,Seconds,TickCount
Minutes,,,
1970-01-01 00:01:00,1970-01-01 00:01:00,1970-01-01 00:00:30,1
1970-01-01 00:01:00,1970-01-01 00:01:00,1970-01-01 00:00:32,1
1970-01-01 00:01:00,1970-01-01 00:01:00,1970-01-01 00:00:50,1
1970-01-01 00:02:00,1970-01-01 00:02:00,1970-01-01 00:00:02,1
1970-01-01 00:02:00,1970-01-01 00:02:00,1970-01-01 00:00:04,1
1970-01-01 00:02:00,1970-01-01 00:02:00,1970-01-01 00:00:22,1
1970-01-01 00:03:00,1970-01-01 00:03:00,1970-01-01 00:00:47,1
1970-01-01 00:03:00,1970-01-01 00:03:00,1970-01-01 00:00:48,1
1970-01-01 00:04:00,1970-01-01 00:04:00,1970-01-01 00:00:02,1


C:\Users\pebaqu\AppData\Roaming\Python\Python36\site-packages\ipykernel_launcher.py:7: FutureWarning: 'Minutes' is both an index level and a column label.
Defaulting to column, but this will raise an ambiguity error in a future version
  import sys


TickCount
Minutes             Seconds                       
1970-01-01 00:00:00 1970-01-01 00:00:00       3555
                    1970-01-01 00:00:01       2426
                    1970-01-01 00:00:02       2016
                    1970-01-01 00:00:03       1605
                    1970-01-01 00:00:04       1614
                    1970-01-01 00:00:05       1519
                    1970-01-01 00:00:06       1475
                    1970-01-01 00:00:07       1401
                    1970-01-01 00:00:08       1218
                    1970-01-01 00:00:09       1318
                    1970-01-01 00:00:10       1350
                    1970-01-01 00:00:11       1344
                    1970-01-01 00:00:12       1227
                    1970-01-01 00:00:13       1174
                    1970-01-01 00:00:14       1228
                    1970-01-01 00:00:15       1318
                    1970-01-01 00:00:16       1282
                    1970-01-01 00:00:17       1129
                    1970-01-01 00:00:18       1116
                    1970-01-01 00:00:19       1133
                    1970-01-01 00:00:20       1323
                    1970-01-01 00:00:21       1024
                    1970-01-01 00:00:22       1194
                    1970-01-01 00:00:23       1162
                    1970-01-01 00:00:24       1159
                    1970-01-01 00:00:25       1162
                    1970-01-01 00:00:26       1149
                    1970-01-01 00:00:27       1112
                    1970-01-01 00:00:28       1042
                    1970-01-01 00:00:29        928
...                                            ...
1970-01-01 00:59:00 1970-01-01 00:00:30       1079
                    1970-01-01 00:00:31        996
                    1970-01-01 00:00:32       1169
                    1970-01-01 00:00:33       1020
                    1970-01-01 00:00:34       1139
                    1970-01-01 00:00:35        994
                    1970-01-01 00:00:36       1025
                    1970-01-01 00:00:37       1069
                    1970-01-01 00:00:38       1050
                    1970-01-01 00:00:39        996
                    1970-01-01 00:00:40       1096
                    1970-01-01 00:00:41        997
                    1970-01-01 00:00:42       1011
                    1970-01-01 00:00:43        946
                    1970-01-01 00:00:44       1037
                    1970-01-01 00:00:45       1050
                    1970-01-01 00:00:46       1097
                    1970-01-01 00:00:47       1102
                    1970-01-01 00:00:48       1060
                    1970-01-01 00:00:49        914
                    1970-01-01 00:00:50       1354
                    1970-01-01 00:00:51       1161
                    1970-01-01 00:00:52       1097
                    1970-01-01 00:00:53       1004
                    1970-01-01 00:00:54       1116
                    1970-01-01 00:00:55       1265
                    1970-01-01 00:00:56       1291
                    1970-01-01 00:00:57       1226
                    1970-01-01 00:00:58       1309
                    1970-01-01 00:00:59       1571

[3600 rows x 1 columns]

In [61]:
df_export = df_analysis[0:750000]

In [30]:
#df_analysis['TTime']=pd.to_datetime(df_analysis['TTime'])

#print(df_analysis.dtypes)

df_analysis['TTime_Week']=df_analysis['TTime'].dt.week
df_analysis['TTime_Day']=df_analysis['TTime'].dt.day
df_analysis['TTime_Hour']=df_analysis['TTime'].dt.hour
df_analysis['TTime_Minute']=df_analysis['TTime'].dt.minute
df_analysis['TTime_Second']=df_analysis['TTime'].dt.second

#display(df_analysis)

display(df_analysis.groupby(df_analysis['TTime_Second'])['TID'].count())
display(df_analysis.groupby(df_analysis['TTime_Minute'])['TID'].count())
display(df_analysis.groupby(df_analysis['TTime_Hour'])['TID'].count())


'''
##DATA EXPORT
df_export = pd.DataFrame(data=arr_test,    # values
columns=arr_analysis.dtype.names)

df_export = df_analysis[0:750000]

#Export sample data
#Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter('C:\\Users\\pebaqu\\Desktop\\Personal\\Python\\JupyterExports\\TickSampling.xlsx', engine='xlsxwriter')

#Convert the dataframe to an XlsxWriter Excel object.
df_export.to_excel(writer, sheet_name='Sheet1')
#Close the Pandas Excel writer and output the Excel file.
writer.save()
##END DATA EXPORT
'''



TTime_Second
0     72727
1     64464
2     59636
3     57359
4     56226
5     55039
6     55229
7     54914
8     53784
9     53459
10    54113
11    53106
12    52958
13    52151
14    51686
15    52805
16    53013
17    52165
18    52126
19    51658
20    52326
21    52269
22    52169
23    52502
24    52636
25    51753
26    51010
27    52008
28    51072
29    51311
30    54490
31    52833
32    52584
33    51599
34    52753
35    52272
36    52409
37    51667
38    51417
39    51529
40    53009
41    51028
42    51270
43    51044
44    50758
45    51282
46    51271
47    50955
48    52018
49    51444
50    52729
51    52254
52    51812
53    51388
54    51722
55    52173
56    51901
57    51551
58    51207
59    53464
Name: TID, dtype: int64

TTime_Minute
0     73220
1     61155
2     55599
3     56861
4     54536
5     54575
6     51545
7     54355
8     51317
9     52545
10    51424
11    50102
12    49179
13    50600
14    53664
15    57127
16    51031
17    50305
18    51039
19    50406
20    50850
21    50341
22    50821
23    51237
24    51170
25    50517
26    48933
27    48662
28    50102
29    50051
30    62989
31    56386
32    53685
33    55066
34    52771
35    54390
36    53185
37    51857
38    53631
39    53352
40    53771
41    51325
42    51036
43    53293
44    52959
45    54735
46    50592
47    49025
48    48729
49    50881
50    52847
51    51147
52    51923
53    51215
54    56155
55    56416
56    50333
57    53951
58    52943
59    61650
Name: TID, dtype: int64

TTime_Hour
0     148941
1     147143
2     130846
3     115816
4      96499
5     103219
6     126475
7     146080
8     152152
9     116960
10    120932
11    103520
12    103335
13    176119
14    190580
15    188949
16    164476
17    108865
18    108350
19    165010
20    127131
21    102315
22    131627
23    110167
Name: TID, dtype: int64

"\n##DATA EXPORT\ndf_export = pd.DataFrame(data=arr_test,    # values\ncolumns=arr_analysis.dtype.names)\n\ndf_export = df_analysis[0:750000]\n\n#Export sample data\n#Create a Pandas Excel writer using XlsxWriter as the engine.\nwriter = pd.ExcelWriter('C:\\Users\\pebaqu\\Desktop\\Personal\\Python\\JupyterExports\\TickSampling.xlsx', engine='xlsxwriter')\n\n#Convert the dataframe to an XlsxWriter Excel object.\ndf_export.to_excel(writer, sheet_name='Sheet1')\n#Close the Pandas Excel writer and output the Excel file.\nwriter.save()\n##END DATA EXPORT\n"

In [92]:
trades = trading_test.activity[1:,]

positive_trades=np.squeeze(np.take(trades, np.where(trades['Profit']>0)))

print(positive_trades['Trade'])

iterator = 0
curr_max_con_win_trade = 0
max_con_win_trade = 0

for i in np.ndenumerate(positive_trades['Trade']):
    if iterator != 0:
        prev_trade = iterator - 1
        if positive_trades[prev_trade]['Trade'] + 1 == positive_trades[iterator]['Trade']:
            curr_max_con_win_trade += 1
            if curr_max_con_win_trade > max_con_win_trade:
                max_con_win_trade = curr_max_con_win_trade
        else:
            curr_max_con_win_trade = 0
        
        #print(positive_trades[max_con_win_trade]['Trade'])
    
    iterator += 1

print(max_con_win_trade)


negative_trades=np.squeeze(np.take(trades, np.where(trades['Profit']<0)))

print(negative_trades['Trade'])

iterator = 0
curr_max_con_lose_trade = 0
max_lose_win_trade = 0

for i in np.ndenumerate(negative_trades['Trade']):
    if iterator != 0:
        prev_trade = iterator - 1
        if negative_trades[prev_trade]['Trade'] + 1 == negative_trades[iterator]['Trade']:
            curr_max_con_lose_trade += 1
            if curr_max_con_lose_trade > max_lose_win_trade:
                max_lose_win_trade = curr_max_con_lose_trade
        else:
            curr_max_con_lose_trade = 0
        
        #print(positive_trades[max_con_win_trade]['Trade'])
    
    iterator += 1

print(max_lose_win_trade)




'''
print(positive_trades)
print(positive_trades.size)

trades[np.where(trades['Profit']>0)]
print(trades)

print(trades)
'''

[    2     4     8 ... 12168 12172 12174]
5
[    1     3     5 ... 12170 12171 12173]
23


"\nprint(positive_trades)\nprint(positive_trades.size)\n\ntrades[np.where(trades['Profit']>0)]\nprint(trades)\n\nprint(trades)\n"

In [38]:
#chart the results
df_results = pd.DataFrame(data=arr_results,    # values
            columns=arr_results.dtype.names)  # 1st row as the column names

df_results.index = df_results['TestName']

display(df_results)

test=df_results.iloc[1:,1:].T

display(test)

,TestName,TotalNetProfit,TotalTrades,GrossProfit,GrossLoss,ProfitFactor,PercentProfitable,WinningTrades,LosingTrades,EvenTrades,...,AvgWinningTrade,AvgLosingTrade,RatioAvgWinAvgLoss,LargestWinningTrade,LargestLosingTrade,MaxConWinTrade,MaxConLoseTrade,AvgBarsInTotalTrades,AvgBarsInWinTrades,AvgBarsInLosTrades
TestName,,,,,,,,,,,,,,,,,,,,,
b'',b'',0.000,0,0.000,0.000,0.000000,0.000000,0,0,0,...,0.000000,0.000000,0.000000,0.000,0.000,0,0,0.000000,0.000000,0.000000
b'test0',b'test0',-302.703,12174,544.306,-847.009,-0.642621,0.230902,2811,9215,148,...,0.193634,-0.091916,-2.106636,2.394,-2.673,5,23,173.015361,420.895411,96.199240
b'test1',b'test1',-59.104,1367,171.984,-231.088,-0.744236,0.261887,358,1003,6,...,0.480402,-0.230397,-2.085108,4.230,-2.232,4,18,1282.381127,2966.064246,680.584247
b'test2',b'test2',-20.946,147,49.050,-69.996,-0.700754,0.204082,30,117,0,...,1.635000,-0.598256,-2.732942,4.914,-1.575,1,13,10448.925170,28400.333333,5846.000000


TestName,b'test0',b'test1',b'test2'
TotalNetProfit,-302.703000,-59.104000,-20.946000
TotalTrades,12174.000000,1367.000000,147.000000
GrossProfit,544.306000,171.984000,49.050000
GrossLoss,-847.009000,-231.088000,-69.996000
ProfitFactor,-0.642621,-0.744236,-0.700754
PercentProfitable,0.230902,0.261887,0.204082
WinningTrades,2811.000000,358.000000,30.000000
LosingTrades,9215.000000,1003.000000,117.000000
EvenTrades,148.000000,6.000000,0.000000
AvgTradeNetProfit,-0.024865,-0.043236,-0.142490


In [15]:
%%time
#create trade report

trades = trading_test.activity[1:,]

positive_trades = np.squeeze(np.take(trades, np.where(trades['Profit']>0)))
negative_trades = np.squeeze(np.take(trades, np.where(trades['Profit']<0)))
even_trades = np.squeeze(np.take(trades, np.where(trades['Profit']==0)))



###### get longest winning and losing trade streak ######

iterator = 0
curr_max_con_win_trade = 0
max_con_win_trade = 0

for i in np.ndenumerate(positive_trades['Trade']):
    if iterator != 0:
        prev_trade = iterator - 1
        if positive_trades[prev_trade]['Trade'] + 1 == positive_trades[iterator]['Trade']:
            curr_max_con_win_trade += 1
            if curr_max_con_win_trade > max_con_win_trade:
                max_con_win_trade = curr_max_con_win_trade
        else:
            curr_max_con_win_trade = 0
              
    iterator += 1


iterator = 0
curr_max_con_lose_trade = 0
max_con_lose_trade = 0

for i in np.ndenumerate(negative_trades['Trade']):
    if iterator != 0:
        prev_trade = iterator - 1
        if negative_trades[prev_trade]['Trade'] + 1 == negative_trades[iterator]['Trade']:
            curr_max_con_lose_trade += 1
            if curr_max_con_lose_trade > max_con_lose_trade:
                max_con_lose_trade = curr_max_con_lose_trade
        else:
            curr_max_con_lose_trade = 0
            
    iterator += 1

##### build arrays #####
arr_results = np.zeros((1,), dtype=[('TestName', 'S28'), ('TotalNetProfit', 'float64'), 
                                    ('TotalTrades', 'int'), ('GrossProfit', 'float64'),
                                    ('GrossLoss', 'float64'), ('ProfitFactor', 'float64'),
                                    ('PercentProfitable', 'float64'),('WinningTrades', 'int'),
                                    ('LosingTrades', 'int'), ('EvenTrades', 'int'),
                                    ('AvgTradeNetProfit', 'float64'), ('AvgWinningTrade', 'float64'),
                                    ('AvgLosingTrade', 'float64'),('RatioAvgWinAvgLoss', 'float64'),
                                    ('LargestWinningTrade', 'float64'), ('LargestLosingTrade', 'float64'),
                                    ('MaxConWinTrade', 'int'),('MaxConLoseTrade', 'int'),
                                    ('AvgBarsInTotalTrades', 'float64'), ('AvgBarsInWinTrades', 'float64'),
                                    ('AvgBarsInLosTrades', 'float64')
                                   ])

arr_new_results = np.empty_like(arr_results)


##### populate arr_new_results #####
arr_new_results['TestName'] = trading_test.name
arr_new_results['TotalNetProfit'] = trading_test.profit
arr_new_results['TotalTrades'] = trades.size #have to subtract the first 0 rowfrom np.zero
arr_new_results['GrossProfit'] = positive_trades['Profit'].sum()
arr_new_results['GrossLoss'] = negative_trades['Profit'].sum()
arr_new_results['ProfitFactor'] = positive_trades['Profit'].sum()/negative_trades['Profit'].sum()
arr_new_results['PercentProfitable'] = positive_trades.size/(trades.size)
arr_new_results['WinningTrades'] = positive_trades.size
arr_new_results['LosingTrades'] = negative_trades.size
arr_new_results['EvenTrades'] = even_trades.size
arr_new_results['AvgTradeNetProfit'] = trading_test.profit/trades.size
arr_new_results['AvgWinningTrade'] = positive_trades['Profit'].sum()/positive_trades.size
arr_new_results['AvgLosingTrade'] = negative_trades['Profit'].sum()/negative_trades.size
arr_new_results['RatioAvgWinAvgLoss'] = (positive_trades['Profit'].sum()/positive_trades.size)/(negative_trades['Profit'].sum()/negative_trades.size)
arr_new_results['LargestWinningTrade'] = np.amax(positive_trades['Profit'])
arr_new_results['LargestLosingTrade'] = np.amin(negative_trades['Profit'])
arr_new_results['MaxConWinTrade'] = max_con_win_trade
arr_new_results['MaxConLoseTrade'] = max_con_lose_trade
arr_new_results['AvgBarsInTotalTrades'] = (trades['Bid_TID'].sum() - trades['Ask_TID'].sum())/trades.size
arr_new_results['AvgBarsInWinTrades'] = (positive_trades['Bid_TID'].sum() - positive_trades['Ask_TID'].sum())/positive_trades.size
arr_new_results['AvgBarsInLosTrades'] = (negative_trades['Bid_TID'].sum() - negative_trades['Ask_TID'].sum())/negative_trades.size

#arr_new_results = 
print(arr_new_results)
#print(arr_results)


#chart the results
df_results = pd.DataFrame(data=arr_new_results,    # values
            columns=arr_new_results.dtype.names)  # 1st row as the column names

df_results.index = df_results['TestName']

display(df_results)

test=df_results.iloc[:,1:].T

display(test)

'''
                              
                              
                              
                              ('GrossProfit', 'float64'),
                                    ('GrossLoss', 'float64'), ('ProfitFactor', 'float64'),
                                    ('PercentProfitable', 'float64'),('WinningTrades', 'int'),
                                    ('LosingTrades', 'int'), ('EvenTrades', 'int'),
                                    ('AvgTradeNetProfit', 'float64'), ('AvgWinningTrade', 'float64'),
                                    ('AvgLosingTrade', 'float64'),('RatioAvgWinAvgLoss', 'float64'),
                                    ('LargestWinningTrade', 'float64'), ('LargestLosingTrade', 'float64'),
                                    ('MaxConWinTrade', 'int'),('MaxConLoseTrade', 'int'),
                                    ('AvgBarsInTotalTrades', 'float64'), ('AvgBarsInWinTrades', 'float64'),
                                    ('AvgBarsInLosTrades', 'float64')')
'''

[(b'test4SMA200000FMA10000SL1', 13.556, 23, 14.186, -0.630, -22.517, 0.522, 12, 11, 0, 0.589, 1.182, -0.057, -20.641, 2.064, -0.279, 1, 2, 8938.913, 16989.833, 156.091)]


,TestName,TotalNetProfit,TotalTrades,GrossProfit,GrossLoss,ProfitFactor,PercentProfitable,WinningTrades,LosingTrades,EvenTrades,...,AvgWinningTrade,AvgLosingTrade,RatioAvgWinAvgLoss,LargestWinningTrade,LargestLosingTrade,MaxConWinTrade,MaxConLoseTrade,AvgBarsInTotalTrades,AvgBarsInWinTrades,AvgBarsInLosTrades
TestName,,,,,,,,,,,,,,,,,,,,,
b'test4SMA200000FMA10000SL1',b'test4SMA200000FMA10000SL1',13.556,23,14.186,-0.63,-22.51746,0.521739,12,11,0,...,1.182167,-0.057273,-20.641005,2.064,-0.279,1,2,8938.913043,16989.833333,156.090909


TestName,b'test4SMA200000FMA10000SL1'
TotalNetProfit,13.556000
TotalTrades,23.000000
GrossProfit,14.186000
GrossLoss,-0.630000
ProfitFactor,-22.517460
PercentProfitable,0.521739
WinningTrades,12.000000
LosingTrades,11.000000
EvenTrades,0.000000
AvgTradeNetProfit,0.589391


Wall time: 46.6 ms


In [103]:
%%time

#convert activity array to dataframe for graphing

df_activity = pd.DataFrame(data=trading_test.activity[1:,],    # values
            columns=trading_test.activity.dtype.names)  # 1st row as the column names

df_activity['Bars'] = df_activity['Bid_TID'] - df_activity['Ask_TID']

display(df_activity['Bars'].sum()/df_activity['Trade'].max())
display(df_activity['Bars'].sum())
display(df_activity)

173.01536060456712

2106289

,Trade,Ask,Ask_TID,Units,Bid,Bid_TID,Profit,Stop_Loss_Floor,Stop_Loss_Bid,Stop_Loss_Bid_TID,Max_DD,Max_DD_TID,Max_DD_Bid,Stop_Loss_Max_DD,Stop_Loss_Max_DD_TID,Stop_Loss_Max_DD_Bid,Bars
0,1,112.661,200,8,112.603,5325,-0.464,0.0,0.0,0,-0.728,1456,112.570,-0.728,1456,112.570,5125
1,2,112.610,5344,8,112.653,8022,0.344,0.0,0.0,0,-0.088,5395,112.599,-0.088,5395,112.599,2678
2,3,112.672,8042,8,112.650,8136,-0.176,0.0,0.0,0,-0.224,8132,112.644,-0.224,8132,112.644,94
3,4,112.649,8218,8,112.735,11003,0.688,0.0,0.0,0,-0.176,8504,112.627,-0.176,8504,112.627,2785
4,5,112.740,11023,8,112.735,11043,-0.040,0.0,0.0,0,-0.048,11041,112.734,-0.048,11041,112.734,20
5,6,112.740,11046,8,112.729,11097,-0.088,0.0,0.0,0,-0.088,11095,112.729,-0.088,11095,112.729,51
6,7,112.733,11185,8,112.726,11196,-0.056,0.0,0.0,0,-0.056,11185,112.726,-0.056,11185,112.726,11
7,8,112.733,11204,8,112.740,12826,0.056,0.0,0.0,0,-0.096,12291,112.721,-0.096,12291,112.721,1622
8,9,112.752,12841,8,112.747,13131,-0.040,0.0,0.0,0,-0.072,13115,112.743,-0.072,13115,112.743,290
9,10,112.751,13137,8,112.747,13145,-0.032,0.0,0.0,0,-0.048,13138,112.745,-0.048,13138,112.745,8


Wall time: 67.2 ms


In [76]:
%%time
#Maximum Adverse Event - for debugging

positive1 = df_activity[df_activity['Profit']>=0]
positive = positive1[['Trade','Max_DD','Profit']].values
negative1 = df_activity[df_activity['Profit']<-.08]
negative = negative1[['Trade','Max_DD','Profit']].values

f, ax = plt.subplots(1)
'''
for i in range(len(positive)):
    pointRefNumber = int(positive[i][0])
    xPoint =  abs(positive[i][1])
    yPoint =  abs(positive[i][2])
    ax.plot(xPoint, yPoint, marker="+", c="g")
    ax.annotate(pointRefNumber, (xPoint, yPoint), fontsize=12)
'''
for i in range(len(negative)):
    pointRefNumber = int(negative[i][0])
    xPoint =  abs(negative[i][1])
    yPoint =  abs(negative[i][2])
    ax.plot(xPoint, yPoint, marker="o", c="r")
    ax.annotate(pointRefNumber, (xPoint, yPoint), fontsize=12)
ax.plot(min(ax.get_ylim(),ax.get_xlim()) , min(ax.get_ylim(),ax.get_xlim()), ls="--", c=".3")
plt.axvline(x=test2.stop_loss)
ax.grid(color='g', linestyle='dashed', linewidth=1)
ax.set_xlim(xmin=0)
ax.set_ylim(ymin=0)
ax.set_title("Maximum Adverse Event \n Stop Loss:" +str(test2.stop_loss))


TypeError: '>' not supported between instances of 'float' and 'NoneType'

In [82]:
%%time

#Maximum Adverse Event

positive = df_activity[df_activity['Profit']>=0]
negative = df_activity[df_activity['Profit']<0]

f, ax = plt.subplots(figsize=(20,10))
ax.scatter(abs(positive['Stop_Loss_Max_DD']), abs(positive['Profit']), marker="+", c="g")
ax.scatter(abs(negative['Stop_Loss_Max_DD']), abs(negative['Profit']), marker="o", c="r")

ax.plot(min(ax.get_ylim(),ax.get_xlim()) , min(ax.get_ylim(),ax.get_xlim()), ls="--", c=".3")
ax.grid(color='g', linestyle='dashed', linewidth=1)
ax.set_xlim(xmin=0)
ax.set_ylim(ymin=0)
plt.axvline(x=test2.stop_loss)
plt.xlabel('Max Drawdown')
plt.ylabel('Profit ($)')
ax.set_title("Maximum Adverse Event \n Stop Loss:" +str(test2.stop_loss))
plt.savefig('C:\\Users\\pebaqu\\Desktop\Personal\\Python\\JupyterExports\\TestGraph.png')

Wall time: 367 ms


In [94]:
%%time
#Underwater Equity Curve

plt.plot(df_activity['Trade'],df_activity['Profit'], color='none', linewidth=0)
plt.fill_between(df_activity['Trade'], df_activity['Profit'], where=df_activity['Profit']>=0, facecolor='green', interpolate=True)
plt.fill_between(df_activity['Trade'], df_activity['Profit'], where=df_activity['Profit']<0, facecolor='red', interpolate=True)
plt.axhline(y=-.1)
plt.title("Underwater Equity Curve \n Stop Loss:" +str(test2.stop_loss))
plt.show

Wall time: 444 ms


In [86]:
display(df_activity.sort_values(by=['Profit']))

,Trade,Ask,Ask_TID,Units,Bid,Bid_TID,Profit,Stop_Loss_Floor,Stop_Loss_Bid,Stop_Loss_Bid_TID,Max_DD,Max_DD_TID,Max_DD_Bid,Stop_Loss_Max_DD,Stop_Loss_Max_DD_TID,Stop_Loss_Max_DD_Bid,Bars
1568,1569,113.134,452833,8,113.021,452855,-0.904,113.121500,0.0,0,-0.976,452833,113.012,-0.976,452833,113.012,22
1382,1383,113.128,402846,8,113.016,402875,-0.896,113.115500,0.0,0,-0.960,402861,113.008,-0.960,402861,113.008,29
4932,4933,110.457,1359807,9,110.375,1359967,-0.738,110.445889,0.0,0,-0.738,1359967,110.375,-0.738,1359967,110.375,160
356,357,112.332,116486,8,112.242,116537,-0.720,112.319500,0.0,0,-0.784,116531,112.234,-0.784,116531,112.234,51
9989,9990,108.767,2646102,9,108.703,2646107,-0.576,108.755889,0.0,0,-0.594,2646106,108.701,-0.594,2646106,108.701,5
12010,12011,109.258,3137962,9,109.199,3137975,-0.531,109.246889,0.0,0,-0.711,3137967,109.179,-0.711,3137967,109.179,13
12008,12009,109.266,3137785,9,109.207,3137841,-0.531,109.254889,0.0,0,-0.531,3137841,109.207,-0.531,3137841,109.207,56
12156,12157,109.266,3177292,9,109.207,3180946,-0.531,109.254889,0.0,0,-0.594,3180933,109.200,-0.594,3180933,109.200,3654
361,362,112.281,121285,8,112.215,121330,-0.528,112.268500,0.0,0,-0.528,121330,112.215,-0.528,121330,112.215,45
12009,12010,109.259,3137883,9,109.204,3137910,-0.495,109.247889,0.0,0,-0.531,3137901,109.200,-0.531,3137901,109.200,27


In [51]:
#understanding stop loss behavior

df_stop_loss = df_activity[df_activity['Max_DD']<= -.1]

display(df_stop_loss)

,Trade,Ask,Ask_TID,Units,Bid,Bid_TID,Profit,Stop_Loss_Floor,Stop_Loss_Bid,Stop_Loss_Bid_TID,Max_DD,Max_DD_TID,Max_DD_Bid,Bars
0,1,112.661,200,8,112.603,5325,-0.096,112.648500,112.649,398,-0.728,1456,112.570,5125
2,3,112.672,8042,8,112.650,8136,-0.072,112.659500,112.663,8042,-0.224,8132,112.644,94
3,4,112.649,8218,8,112.735,11003,-0.096,112.636500,112.637,8222,-0.176,8504,112.627,2785
17,18,112.785,15021,8,112.772,15360,-0.040,112.772500,112.780,15021,-0.104,15350,112.772,339
18,19,112.777,15393,8,112.760,15922,-0.040,112.764500,112.772,15393,-0.152,15921,112.758,529
19,20,112.780,15948,8,112.759,16297,-0.040,112.767500,112.775,15948,-0.176,16292,112.758,349
20,21,112.764,16394,8,112.754,16547,-0.056,112.751500,112.757,16394,-0.104,16540,112.751,153
23,24,112.753,17540,8,112.738,17601,-0.032,112.740500,112.749,17540,-0.120,17601,112.738,61
26,27,112.739,19575,8,112.729,19620,-0.032,112.726500,112.735,19575,-0.104,19616,112.726,45
28,29,112.737,19685,8,112.715,19743,-0.064,112.724500,112.729,19685,-0.176,19743,112.715,58


In [48]:
#understanding stop loss behavior

df_stop_loss = df_activity[df_activity['Max_DD']<= -.1]

display(df_stop_loss)

,Trade,Ask,Ask_TID,Units,Bid,Bid_TID,Profit,Stop_Loss_Floor,Stop_Loss_Bid,Stop_Loss_Bid_TID,Max_DD,Max_DD_TID,Max_DD_Bid,Bars
0,1,112.661,200,8,112.603,5325,-0.464,0.0,0.0,0,-0.728,1456,112.570,5125
2,3,112.672,8042,8,112.650,8136,-0.176,0.0,0.0,0,-0.224,8132,112.644,94
3,4,112.649,8218,8,112.735,11003,0.688,0.0,0.0,0,-0.176,8504,112.627,2785
17,18,112.785,15021,8,112.772,15360,-0.104,0.0,0.0,0,-0.104,15350,112.772,339
18,19,112.777,15393,8,112.760,15922,-0.136,0.0,0.0,0,-0.152,15921,112.758,529
19,20,112.780,15948,8,112.759,16297,-0.168,0.0,0.0,0,-0.176,16292,112.758,349
20,21,112.764,16394,8,112.754,16547,-0.080,0.0,0.0,0,-0.104,16540,112.751,153
23,24,112.753,17540,8,112.738,17601,-0.120,0.0,0.0,0,-0.120,17601,112.738,61
26,27,112.739,19575,8,112.729,19620,-0.080,0.0,0.0,0,-0.104,19616,112.726,45
28,29,112.737,19685,8,112.715,19743,-0.176,0.0,0.0,0,-0.176,19743,112.715,58


In [5]:
#troubleshooting "too many indices error" test trade 358 (should not produce error)

start = 116563 - 200
stop = 118729 - 200

ask = 112.304
units = 8
stop_loss = .1
equity = units * ask

print("equity is:", equity)


arr_test = arr_analysis[start:stop]


max_dd_criteria = (equity-stop_loss)/units #calculate the Bid value of your loss stop
print("max_dd_criteria is:" , max_dd_criteria)


max_dd_min_bid = np.empty_like(arr_analysis) #create the empty array for your analysis

max_dd_min_bid = arr_analysis[start:stop] #fill the array with your start and stop
print("max_dd_min_bid dimensions are:" , max_dd_min_bid.ndim)

#need to apply a test to see what the shape is after the squeeze
arr_max_dd_min_bid = np.squeeze(np.take(max_dd_min_bid, np.where(max_dd_min_bid['Bid']>= max_dd_criteria)))
print("arr_max_dd_min_bid is:" , arr_max_dd_min_bid)
print("arr_max_dd_min_bid shape is:", arr_max_dd_min_bid.shape)
print("arr_max_dd_min_bid size:", arr_max_dd_min_bid.size)


#handle data sets where bid is never >= min bid
if arr_max_dd_min_bid.size == 0:
    arr_max_dd_min_bid = max_dd_min_bid[np.argmax(max_dd_min_bid['Bid'])]
    min_bid = arr_max_dd_min_bid['Bid']
    min_bid_tid = arr_max_dd_min_bid['TID']

else:


    min_bid = np.amin(arr_max_dd_min_bid['Bid']) #the lowest purchase price in the range of your purchase to sale
    print("min bid is:", min_bid)

    min_index = int(np.argmin(arr_max_dd_min_bid['Bid']))
    print("min_index is:", min_index)
    min_bid_tuple = arr_max_dd_min_bid[min_index] #the row of the lowest purchase price in the range of your purchase to sale
    min_bid_tid = min_bid_tuple['TID'] #the ID of the lowest purchase price


equity is: 898.432
max_dd_criteria is: 112.2915
max_dd_min_bid dimensions are: 1
arr_max_dd_min_bid is: [('USD/JPY', '2018-01-02T22:24:31.428000000', 112.304, 112.33 , 116585, -0.026, 112.29999 , 112.300085, 112.28725, 112.2859 , 112.318115, 112.31812 , 112.31025, 112.30895, -0.018125, -0.018035, -0.023  , -0.02305)
 ('USD/JPY', '2018-01-02T22:24:31.481000000', 112.304, 112.334, 116586, -0.03 , 112.299915, 112.29999 , 112.28855, 112.28725, 112.31813 , 112.318115, 112.31175, 112.31025, -0.018215, -0.018125, -0.0232 , -0.023  )
 ('USD/JPY', '2018-01-02T22:24:31.752000000', 112.315, 112.334, 116587, -0.019, 112.299875, 112.299915, 112.29035, 112.28855, 112.318145, 112.31813 , 112.31325, 112.31175, -0.01827 , -0.018215, -0.0229 , -0.0232 )
 ('USD/JPY', '2018-01-02T22:24:31.754000000', 112.304, 112.334, 116588, -0.03 , 112.2998  , 112.299875, 112.2914 , 112.29035, 112.31816 , 112.318145, 112.31475, 112.31325, -0.01836 , -0.01827 , -0.02335, -0.0229 )
 ('USD/JPY', '2018-01-02T22:24:32.130000

In [ ]:
df_activity[355:360]

In [21]:
#troubleshooting "too many indices error" trade 357
trade = 1383
start = 402846 - 200
stop = 402876 - 200

ask = 113.128
units = 8
stop_loss = .1
equity = units * ask

print("equity is:", equity)


arr_test = arr_analysis[start:stop]

#max_dd_criteria
stop_loss_bid = (equity-stop_loss)/units #calculate the Bid value of your loss stop
print("stop loss bid is:" , stop_loss_bid)


arr_sub_analysis = np.empty_like(arr_analysis) #create the empty array for your analysis

arr_sub_analysis = arr_analysis[start:stop] #fill the array with your start and stop
print("arr_sub_analysis dimensions are:" , arr_sub_analysis.ndim)

#need to apply a test to see what the shape is after the squeeze
arr_gte_stop_loss = np.squeeze(np.take(arr_sub_analysis, np.where(arr_sub_analysis['Bid']>= stop_loss_bid)))
print("arr_gte_stop_loss is:" , arr_gte_stop_loss)
print("arr_gte_stop_loss shape is:", arr_gte_stop_loss.shape)
print("arr_gte_stop_loss size:", arr_gte_stop_loss.size)


#handle data sets where bid is never >= min bid. We actuially want to skip stop loss and just use standard logic for realism
if arr_gte_stop_loss.size < 2:
    arr_stop_loss = arr_sub_analysis[np.argmin(arr_sub_analysis['Bid'])] #take the lowest bid for worst case scenario
    stop_loss_bid = arr_stop_loss['Bid']
    print("single tuple stop_loss is:", stop_loss_bid)
    min_bid_tid = arr_stop_loss['TID']

else:

    stop_loss = np.amin(arr_gte_stop_loss['Bid']) #the lowest purchase price in the range of your purchase to sale
    print("multiple tuple stop loss is:", stop_loss)

    stop_loss_bid_index = np.argmin(arr_gte_stop_loss['Bid'])
    print("multiple tuple stop loss index is:", stop_loss_bid_index)
    arr_stop_loss_bid = arr_gte_stop_loss[stop_loss_bid_index] #the row of the lowest purchase price in the range of your purchase to sale
    stop_loss_bid_tid = arr_stop_loss_bid['TID'] #the ID of the lowest purchase price


##DATA EXPORT
df_export = pd.DataFrame(data=arr_test,    # values
columns=arr_analysis.dtype.names)

#Export sample data
#Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter('C:\\Users\\pebaqu\\Desktop\\Personal\\Python\\JupyterExports\\Trade '+str(trade)+' analysis.xlsx', engine='xlsxwriter')

#Convert the dataframe to an XlsxWriter Excel object.
df_export.to_excel(writer, sheet_name='Sheet1')
#Close the Pandas Excel writer and output the Excel file.
writer.save()
##END DATA EXPORT


equity is: 905.024
stop loss bid is: 113.1155
arr_sub_analysis dimensions are: 1
arr_gte_stop_loss is: []
arr_gte_stop_loss shape is: (0,)
arr_gte_stop_loss size: 0
single tuple stop_loss is: 113.008


In [20]:
start = 115723 - 200 #have to set the array index to 200 less because of the drop na
stop = 120934 - 200

df_export = pd.DataFrame(data=arr_analysis[0:100],    # values
            columns=arr_analysis.dtype.names)
        
#Export sample data
#Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter('C:\\Users\\pebaqu\\Desktop\\Personal\\Python\\JupyterExports\\Test.xlsx', engine='xlsxwriter')

#Convert the dataframe to an XlsxWriter Excel object.
df_export.to_excel(writer, sheet_name='Sheet1')
#Close the Pandas Excel writer and output the Excel file.
writer.save()
##END DATA EXPORTarr_analysis[115723:120936]

In [13]:
df_argmin = pd.DataFrame(data=arr_analysis[118776:118779,],    # values
            columns=arr_analysis.dtype.names)

display(df_argmin)

arr_sub_argmin = arr_analysis[118772:118775,]

print(arr_sub_argmin)

#test = np.empty_like(arr_analysis)

np.argmin(arr_sub_argmin['Bid'])
#print(test)

,Pair,TTime,Bid,Ask,TID,Bid_Ask_Spread,Bid_Fast_Avg,Prev_Bid_Fast_Avg,Bid_Slow_Avg,Prev_Bid_Slow_Avg,Ask_Fast_Avg,Prev_Ask_Fast_Avg,Ask_Slow_Avg,Prev_Ask_Slow_Avg,Spread_Fast_Avg,Prev_Spread_Fast_Avg,Spread_Slow_Avg,Prev_Spread_Slow_Avg
0,USD/JPY,2018-01-02 22:36:38.505,112.290,112.304,118976,-0.014,112.26965,112.269535,112.2880,112.28725,112.292255,112.292160,112.30170,112.3017,-0.022605,-0.022625,-0.01370,-0.01445
1,USD/JPY,2018-01-02 22:36:38.557,112.289,112.304,118977,-0.015,112.26976,112.269650,112.2885,112.28800,112.292345,112.292255,112.30190,112.3017,-0.022585,-0.022605,-0.01340,-0.01370
2,USD/JPY,2018-01-02 22:36:38.823,112.289,112.303,118978,-0.014,112.26987,112.269760,112.2885,112.28850,112.292425,112.292345,112.30205,112.3019,-0.022555,-0.022585,-0.01355,-0.01340


[('USD/JPY', '2018-01-02T22:36:35.910000000', 112.289, 112.303, 118972, -0.014, 112.269, 112.269, 112.286, 112.285, 112.292, 112.292, 112.301, 112.301, -0.023, -0.023, -0.015, -0.016)
 ('USD/JPY', '2018-01-02T22:36:36.758000000', 112.289, 112.304, 118973, -0.015, 112.269, 112.269, 112.286, 112.286, 112.292, 112.292, 112.301, 112.301, -0.023, -0.023, -0.015, -0.015)
 ('USD/JPY', '2018-01-02T22:36:36.809000000', 112.289, 112.303, 118974, -0.014, 112.269, 112.269, 112.287, 112.286, 112.292, 112.292, 112.302, 112.301, -0.023, -0.023, -0.015, -0.015)]


0

In [15]:
df_argmin = pd.DataFrame(data=arr_analysis[118772:118775,],    # values
            columns=arr_analysis.dtype.names)

display(df_argmin)

arr_sub_argmin = arr_analysis[118772:118775,]

print(arr_sub_argmin)

#test = np.empty_like(arr_analysis)

np.argmin(arr_sub_argmin['Bid'])
#print(test)

min_bid_tuple = arr_sub_argmin[np.argmin(arr_sub_argmin['Bid'])]
print(min_bid_tuple)


'''
min_bid_tuple = arr_max_dd_min_bid[np.argmin(arr_max_dd_min_bid['Bid'])]
too many indices for the array
'''







max_dd_criteria = 112.2775 #calculate the Bid value of your loss stop


max_dd_min_bid = np.empty_like(arr_analysis) #create the empty array for your analysis

max_dd_min_bid = arr_analysis[118772:118775] #fill the array with your start and stop
'''
##DATA EXPORT
df_export = pd.DataFrame(data=max_dd_min_bid,    # values
columns=max_dd_min_bid.dtype.names)

#Export sample data
#Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter('C:\\Users\\pebaqu\\Desktop\\Personal\\Python\\JupyterExports\\max_dd_min_bid' + str(self.trades) + '.xlsx', engine='xlsxwriter')

#Convert the dataframe to an XlsxWriter Excel object.
df_export.to_excel(writer, sheet_name='Sheet1')
#Close the Pandas Excel writer and output the Excel file.
writer.save()
##END DATA EXPORT
'''

arr_max_dd_min_bid = np.squeeze(np.take(max_dd_min_bid, np.where(max_dd_min_bid['Bid']>= max_dd_criteria)))
print(arr_max_dd_min_bid)

'''

##DATA EXPORT
df_export = pd.DataFrame.from_records(data=arr_max_dd_min_bid)#,    # values
#columns=arr_max_dd_min_bid.dtype.names)

#Export sample data
#Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter('C:\\Users\\pebaqu\\Desktop\\Personal\\Python\\JupyterExports\\arr_max_dd_min_bid' + str(self.trades) + '.xlsx', engine='xlsxwriter')

#Convert the dataframe to an XlsxWriter Excel object.
df_export.to_excel(writer, sheet_name='Sheet1')
#Close the Pandas Excel writer and output the Excel file.
writer.save()
##END DATA EXPORT
'''

min_bid = np.amin(arr_max_dd_min_bid['Bid']) #the lowest purchase price in the range of your purchase to sale

min_bid_tuple = arr_max_dd_min_bid[np.argmin(arr_max_dd_min_bid['Bid'])] #the row of the lowest purchase price in the range of your purchase to sale
min_bid_tid = min_bid_tuple['TID'] #the ID of the lowest purchase price




,Pair,TTime,Bid,Ask,TID,Bid_Ask_Spread,Bid_Fast_Avg,Prev_Bid_Fast_Avg,Bid_Slow_Avg,Prev_Bid_Slow_Avg,Ask_Fast_Avg,Prev_Ask_Fast_Avg,Ask_Slow_Avg,Prev_Ask_Slow_Avg,Spread_Fast_Avg,Prev_Spread_Fast_Avg,Spread_Slow_Avg,Prev_Spread_Slow_Avg
0,USD/JPY,2018-01-02 22:36:35.910,112.289,112.303,118972,-0.014,112.269175,112.269055,112.2859,112.28545,112.29191,112.291845,112.30120,112.3011,-0.022735,-0.022790,-0.01530,-0.01565
1,USD/JPY,2018-01-02 22:36:36.758,112.289,112.304,118973,-0.015,112.269295,112.269175,112.2864,112.28590,112.29199,112.291910,112.30140,112.3012,-0.022695,-0.022735,-0.01500,-0.01530
2,USD/JPY,2018-01-02 22:36:36.809,112.289,112.303,118974,-0.014,112.269415,112.269295,112.2868,112.28640,112.29207,112.291990,112.30155,112.3014,-0.022655,-0.022695,-0.01475,-0.01500


[('USD/JPY', '2018-01-02T22:36:35.910000000', 112.289, 112.303, 118972, -0.014, 112.269, 112.269, 112.286, 112.285, 112.292, 112.292, 112.301, 112.301, -0.023, -0.023, -0.015, -0.016)
 ('USD/JPY', '2018-01-02T22:36:36.758000000', 112.289, 112.304, 118973, -0.015, 112.269, 112.269, 112.286, 112.286, 112.292, 112.292, 112.301, 112.301, -0.023, -0.023, -0.015, -0.015)
 ('USD/JPY', '2018-01-02T22:36:36.809000000', 112.289, 112.303, 118974, -0.014, 112.269, 112.269, 112.287, 112.286, 112.292, 112.292, 112.302, 112.301, -0.023, -0.023, -0.015, -0.015)]
('USD/JPY', '2018-01-02T22:36:35.910000000', 112.289, 112.303, 118972, -0.014, 112.269, 112.269, 112.286, 112.285, 112.292, 112.292, 112.301, 112.301, -0.023, -0.023, -0.015, -0.016)


"\nmin_bid_tuple = arr_max_dd_min_bid[np.argmin(arr_max_dd_min_bid['Bid'])]\ntoo many indices for the array\n"

In [16]:
#figure out why trade 357 was bombing out - there was no bid >= to min bid

start = 118772 - 200
stop = 118775 - 200

max_dd_criteria = 112.2775 #calculate the Bid value of your loss stop

arr_max_dd_min_bid = np.squeeze(np.take(max_dd_min_bid, np.where(max_dd_min_bid['Bid']>= max_dd_criteria)))

max_dd_min_bid = np.empty_like(arr_analysis) #create the empty array for your analysis

max_dd_min_bid = arr_analysis[start:stop] #fill the array with your start and stop

arr_max_dd_min_bid = np.squeeze(np.take(max_dd_min_bid, np.where(max_dd_min_bid['Bid']>= max_dd_criteria)))

print(arr_max_dd_min_bid.shape)
print(arr_max_dd_min_bid.strides)
print(arr_max_dd_min_bid.size)
print(max_dd_min_bid[['Bid','Ask','TID']])
print(arr_max_dd_min_bid.ndim, arr_max_dd_min_bid.size)
print(max_dd_min_bid.ndim, max_dd_min_bid.size)
#print(np.argmin(arr_max_dd_min_bid['Bid']))

#min_bid_tuple = arr_max_dd_min_bid[np.argmin(arr_max_dd_min_bid['Bid'])]

'''
if np.squeeze(np.take(max_dd_min_bid, np.where(max_dd_min_bid['Bid']>= max_dd_criteria))).size == 0:
    print(np.argmax(max_dd_min_bid['Bid']))
    arr_max_dd_min_bid = max_dd_min_bid[np.argmax(max_dd_min_bid['Bid'])]
    min_bid = arr_max_dd_min_bid['Bid']
    min_bid_tid = arr_max_dd_min_bid['TID']

else:
    print("pass")

print(min_bid)
print(min_bid_tid)

'''

(0,)
(140,)
0
[(112.265, 112.290, 118772) (112.265, 112.288, 118773)
 (112.265, 112.287, 118774)]
1 0
1 3


'\nif np.squeeze(np.take(max_dd_min_bid, np.where(max_dd_min_bid[\'Bid\']>= max_dd_criteria))).size == 0:\n    print(np.argmax(max_dd_min_bid[\'Bid\']))\n    arr_max_dd_min_bid = max_dd_min_bid[np.argmax(max_dd_min_bid[\'Bid\'])]\n    min_bid = arr_max_dd_min_bid[\'Bid\']\n    min_bid_tid = arr_max_dd_min_bid[\'TID\']\n\nelse:\n    print("pass")\n\nprint(min_bid)\nprint(min_bid_tid)\n\n'